In [1]:
import pyomo.environ as pyo

# Datos de entrada
years = [1, 2, 3, 4, 5]
lands = [1, 2, 3, 4]
ages = list(range(1, 13))
cow_ages = list(range(2, 12))

gr_area = {1: 20.0, 2: 30.0, 3: 20.0, 4: 10.0}
gr_yield = {1: 1.1, 2: 0.9, 3: 0.8, 4: 0.65}
sb_yield = 1.5
housing_cap = 130
gr_intake, sb_intake = 0.6, 0.7
hf_land, land_cap = 2/3.0, 200
hf_labor, cow_labor, gr_labor, sb_labor = 0.1, 0.42, 0.04, 0.14
labor_cap = 55.0
cow_decay, hf_decay = 0.02, 0.05
initial_hf, initial_cows = 9.5, 9.8
birthrate = 1.1
min_final_cows, max_final_cows = 50, 175

# Precios y costos
bl_price, hf_price, cow_price = 30, 40, 120
milk_price, gr_price, sb_price = 370, 75, 58
gr_cost, sb_cost = 90, 70
overtime_cost, regular_time_cost = 120, 4000
hf_cost, cow_cost = 50, 100
gr_land_cost, sb_land_cost = 15, 10
installment = 39.71

# Modelo
model = pyo.ConcreteModel()

# Conjuntos
model.Y = pyo.Set(initialize=years)
model.L = pyo.Set(initialize=lands)
model.A = pyo.Set(initialize=ages)
model.CA = pyo.Set(initialize=cow_ages)

# Variables
model.SB = pyo.Var(model.Y, domain=pyo.NonNegativeReals)
model.GR_buy = pyo.Var(model.Y, domain=pyo.NonNegativeReals)
model.GR_sell = pyo.Var(model.Y, domain=pyo.NonNegativeReals)
model.SB_buy = pyo.Var(model.Y, domain=pyo.NonNegativeReals)
model.SB_sell = pyo.Var(model.Y, domain=pyo.NonNegativeReals)
model.Overtime = pyo.Var(model.Y, domain=pyo.NonNegativeReals)
model.Outlay = pyo.Var(model.Y, domain=pyo.NonNegativeReals)
model.HF_sell = pyo.Var(model.Y, domain=pyo.NonNegativeReals)
model.Newborn = pyo.Var(model.Y, domain=pyo.NonNegativeReals)
model.Profit = pyo.Var(model.Y, domain=pyo.Reals)
model.GR = pyo.Var(model.Y, model.L, domain=pyo.NonNegativeReals)
model.Cows = pyo.Var(model.Y, model.A, domain=pyo.NonNegativeReals)

# Restricciones

def housing_rule(m, t):
    return m.Newborn[t] + m.Cows[t, 1] + sum(m.Cows[t, a] for a in model.CA) <= housing_cap + sum(m.Outlay[d] for d in model.Y if d <= t)
model.HousingCap = pyo.Constraint(model.Y, rule=housing_rule)

def grain_rule(m, t):
    return sum(gr_intake * m.Cows[t, a] for a in model.CA) <= m.GR_buy[t] - m.GR_sell[t] + sum(m.GR[t, l] for l in model.L)
model.GrainCons = pyo.Constraint(model.Y, rule=grain_rule)

def beet_rule(m, t):
    return sum(sb_intake * m.Cows[t, a] for a in model.CA) <= m.SB_buy[t] - m.SB_sell[t] + m.SB[t]
model.BeetCons = pyo.Constraint(model.Y, rule=beet_rule)

def land_rule(m, t):
    return (m.SB[t]/sb_yield + hf_land*(m.Newborn[t] + m.Cows[t, 1]) +
            sum(m.GR[t, l]/gr_yield[l] for l in model.L) + sum(m.Cows[t, a] for a in model.CA)) <= land_cap
model.LandCap = pyo.Constraint(model.Y, rule=land_rule)

def labor_rule(m, t):
    return (hf_labor*(m.Newborn[t] + m.Cows[t,1]) + sum(cow_labor * m.Cows[t,a] for a in model.CA)
            + sum(gr_labor/gr_yield[l]*m.GR[t,l] for l in model.L) + sb_labor/sb_yield*m.SB[t]
           <= labor_cap + m.Overtime[t])
model.LaborCap = pyo.Constraint(model.Y, rule=labor_rule)

def continuity_rule_a(m, t):
    if t == min(years): return pyo.Constraint.Skip
    return m.Cows[t,1] == (1 - hf_decay)*m.Newborn[t-1]
model.Cont1 = pyo.Constraint(model.Y, rule=continuity_rule_a)

def continuity_rule_b(m, t):
    if t == min(years): return pyo.Constraint.Skip
    return m.Cows[t,2] == (1 - hf_decay)*m.Cows[t-1,1]
model.Cont2 = pyo.Constraint(model.Y, rule=continuity_rule_b)

def continuity_rule_c(m, t, a):
    if t == min(years) or a == 1 or a == 12:
        return pyo.Constraint.Skip
    return m.Cows[t,a+1] == (1 - cow_decay)*m.Cows[t-1,a]
model.Cont3 = pyo.Constraint(model.Y, model.CA, rule=continuity_rule_c)

def heifer_rule(m, t):
    return m.Newborn[t] + m.HF_sell[t] == sum((birthrate/2)*m.Cows[t,a] for a in model.CA)
model.HeiferBirth = pyo.Constraint(model.Y, rule=heifer_rule)

model.FinalCows = pyo.Constraint(expr=sum(model.Cows[5,a] for a in model.CA) >= min_final_cows)
model.MaxCows = pyo.Constraint(expr=sum(model.Cows[5,a] for a in model.CA) <= max_final_cows)

# Condiciones iniciales
model.InitHeifers = pyo.ConstraintList()
model.InitCows = pyo.ConstraintList()
for a in ages:
    if a < 3:
        model.InitHeifers.add(model.Cows[1,a] == initial_hf)
    else:
        model.InitCows.add(model.Cows[1,a] == initial_cows)

# Beneficio anual
def profit_rule(m, t):
    return m.Profit[t] == (bl_price*birthrate/2*sum(m.Cows[t,a] for a in model.CA)
                           + hf_price*m.HF_sell[t] + cow_price*m.Cows[t,12]
                           + milk_price*sum(m.Cows[t,a] for a in model.CA)
                           + gr_price*m.GR_sell[t] + sb_price*m.SB_sell[t]
                           - gr_cost*m.GR_buy[t] - sb_cost*m.SB_buy[t]
                           - overtime_cost*m.Overtime[t] - regular_time_cost
                           - hf_cost*(m.Newborn[t] + m.Cows[t,1])
                           - cow_cost*sum(m.Cows[t,a] for a in model.CA)
                           - gr_land_cost*sum(m.GR[t,l]/gr_yield[l] for l in model.L)
                           - sb_land_cost*m.SB[t]/sb_yield
                           - installment*sum(m.Outlay[d] for d in model.Y if d <= t))
model.ProfitCalc = pyo.Constraint(model.Y, rule=profit_rule)

# Función objetivo
model.Obj = pyo.Objective(expr=sum(model.Profit[t] - installment*(t+4)*model.Outlay[t] for t in model.Y), sense=pyo.maximize)

# Resolviendo
solver = pyo.SolverFactory('glpk')  # o 'cbc', 'gurobi' si está disponible
result = solver.solve(model, tee=True)


GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\a_cor\AppData\Local\Temp\tmp9ox7ok10.glpk.raw --wglp C:\Users\a_cor\AppData\Local\Temp\tmpzptqzi07.glpk.glp
 --cpxlp C:\Users\a_cor\AppData\Local\Temp\tmp1m0ddo_v.pyomo.lp
Reading problem data from 'C:\Users\a_cor\AppData\Local\Temp\tmp1m0ddo_v.pyomo.lp'...
97 rows, 130 columns, 723 non-zeros
1163 lines were read
Writing problem data to 'C:\Users\a_cor\AppData\Local\Temp\tmpzptqzi07.glpk.glp'...
1012 lines were written
GLPK Simplex Optimizer, v4.65
97 rows, 130 columns, 723 non-zeros
Preprocessing...
39 rows, 70 columns, 204 non-zeros
Scaling...
 A: min|aij| =  3.636e-02  max|aij| =  1.538e+00  ratio =  4.231e+01
GM: min|aij| =  4.774e-01  max|aij| =  2.095e+00  ratio =  4.387e+00
EQ: min|aij| =  2.279e-01  max|aij| =  1.000e+00  ratio =  4.387e+00
Constructing initial basis...
Size of triangular part is 39
      0: obj =   1.123142665e+05 inf =   3.169e+02 (10)
     10: obj =   6.986564297e